In [1]:
import numpy as np
import cv2
import glob
import os
import yaml

ModuleNotFoundError: No module named 'yaml'

In [2]:
INPUT = './videos/2019-02-05-184551.avi'
DIR = 'corners/'

In [3]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [4]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
pattern_size = (9, 6)
objp = np.zeros((np.prod(pattern_size),3), np.float32)
objp[:,:2] = np.indices(pattern_size).T.reshape(-1,2)

In [5]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

In [6]:
source = cv2.VideoCapture(INPUT)
source.isOpened()

True

In [7]:
f_count = 0
f_step = 10

while True:
         
    # Capture frame-by-frame
    ret, frame = source.read()   

    if not ret: 
        break
        
    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, cv2.CALIB_CB_FILTER_QUADS)
    
    if ret:        
        objpoints.append(objp)
        
        corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria)
        imgpoints.append(corners2)
        
        # Draw and display the corners
        img = cv2.drawChessboardCorners(frame, pattern_size, corners2, ret)
        cv2.imwrite(os.path.join(DIR, '{0}_corner.png'.format(f_count)), img)
        
    f_count += 1

In [8]:
source.release()
cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [9]:
calibration = {'ret': ret, 'mtx': mtx.tolist(), 'dist': dist.tolist() }

In [10]:
with open('calibration.yaml', 'w') as fw:
        yaml.dump(calibration, fw)

In [10]:
calibration

{'ret': 0.5749367191287328,
 'mtx': [[1502.181572345701, 0.0, 446.74251314424254],
  [0.0, 1481.4806325161464, 298.98237501204625],
  [0.0, 0.0, 1.0]],
 'dist': [[-0.6497354207742148,
   1.1908487484072574,
   0.020848568922629356,
   0.06842770962532546,
   -0.6606431134116602]]}